In [1]:
import pickle # For loading model
import difflib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import pickle

In [2]:
!gdown --id 1_n03w2XPQQlbEBuWFHMSZJql3FXXBrTd

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_n03w2XPQQlbEBuWFHMSZJql3FXXBrTd
To: /content/wisata_bali2.csv
100% 33.8k/33.8k [00:00<00:00, 41.1MB/s]


In [3]:
df = pd.read_csv('wisata_bali2.csv')
df

,category,tags,image,weekend_price,weekday_price,distance,rating,place_name,city,description
0,Agrowisata,"Agrowisata, Gianyar, Dekat dari Bandara, Mahal",https://i0.wp.com/sabani.com/wp-content/upload...,50000,50000,dekat,3.0,Agrowisata Satria,Gianyar,Agrowisata Satria menawarkan rasa Bali pada se...
1,Agrowisata,"Agrowisata, Gianyar, Sedang dari Bandara, Mahal",https://static.cityspride.com/img/ruuu_1101/pr...,300000,300000,sedang,5.0,Aloha Ubud Swing,Gianyar,Ajaklah orangorang tercinta untuk mencoba ayun...
2,Agrowisata,"Agrowisata, Denpasar, Dekat dari Bandara, Murah",https://www.rentalmobilbali.net/wp-content/upl...,35000,30000,dekat,4.0,Big Garden Corner,Denpasar,Objek wisata Big Garden Corner Sanur adalah se...
3,Agrowisata,"Agrowisata, Gianyar, Dekat dari Bandara, Gratis",https://www.omnivagant.com/wp-content/uploads/...,0,0,dekat,4.5,Campuhan Ridge Walk,Gianyar,Campuhan Ridge Walk adalah sebuah trek jogging...
4,Agrowisata,"Agrowisata, Tabanan, Sedang dari Bandara, Murah",https://img.freepik.com/premium-photo/green-ri...,40000,40000,sedang,4.5,Jatiluwih Green Land,Tabanan,Nominasi untuk penunjukan sebagai situs Warisa...
...,...,...,...,...,...,...,...,...,...,...
70,Religius,"Religius, Gianyar, Dekat dari Bandara, Gratis",https://liburdulu.com/wp-content/uploads/2020/...,0,0,dekat,4.0,Pura Saraswati,Gianyar,Pura Saraswati merupakan pura yang didirikan u...
71,Religius,"Religius, Badung, Dekat dari Bandara, Murah",https://img.freepik.com/free-photo/pura-taman-...,15000,15000,dekat,4.0,Pura Taman Ayun,Badung,Pura Taman Ayun di susun dengan konsep taman t...
72,Religius,"Religius, Gianyar, Sedang dari Bandara, Murah",https://media.istockphoto.com/id/1355012798/ph...,30000,30000,sedang,4.5,Pura Tirta Empul,Gianyar,Tirta Empul adalah nama sebuah pura Hindu yang...
73,Religius,"Religius, Badung, Dekat dari Bandara, Mahal",https://museumnusantara.com/wp-content/uploads...,50000,50000,dekat,4.0,Pura Uluwatu,Badung,Pura yang terletak di ujung barat daya pulau B...


In [4]:
df_filter = df[['place_name', 'tags', 'description']]
df_filter.head()

,place_name,tags,description
0,Agrowisata Satria,"Agrowisata, Gianyar, Dekat dari Bandara, Mahal",Agrowisata Satria menawarkan rasa Bali pada se...
1,Aloha Ubud Swing,"Agrowisata, Gianyar, Sedang dari Bandara, Mahal",Ajaklah orangorang tercinta untuk mencoba ayun...
2,Big Garden Corner,"Agrowisata, Denpasar, Dekat dari Bandara, Murah",Objek wisata Big Garden Corner Sanur adalah se...
3,Campuhan Ridge Walk,"Agrowisata, Gianyar, Dekat dari Bandara, Gratis",Campuhan Ridge Walk adalah sebuah trek jogging...
4,Jatiluwih Green Land,"Agrowisata, Tabanan, Sedang dari Bandara, Murah",Nominasi untuk penunjukan sebagai situs Warisa...


In [5]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   place_name   75 non-null     object
 1   tags         75 non-null     object
 2   description  75 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


In [6]:
df.shape

(75, 10)

In [7]:
df_filter = df_filter.dropna(axis='rows')
df_filter = df_filter.fillna('unknown')
df_filter

,place_name,tags,description
0,Agrowisata Satria,"Agrowisata, Gianyar, Dekat dari Bandara, Mahal",Agrowisata Satria menawarkan rasa Bali pada se...
1,Aloha Ubud Swing,"Agrowisata, Gianyar, Sedang dari Bandara, Mahal",Ajaklah orangorang tercinta untuk mencoba ayun...
2,Big Garden Corner,"Agrowisata, Denpasar, Dekat dari Bandara, Murah",Objek wisata Big Garden Corner Sanur adalah se...
3,Campuhan Ridge Walk,"Agrowisata, Gianyar, Dekat dari Bandara, Gratis",Campuhan Ridge Walk adalah sebuah trek jogging...
4,Jatiluwih Green Land,"Agrowisata, Tabanan, Sedang dari Bandara, Murah",Nominasi untuk penunjukan sebagai situs Warisa...
...,...,...,...
70,Pura Saraswati,"Religius, Gianyar, Dekat dari Bandara, Gratis",Pura Saraswati merupakan pura yang didirikan u...
71,Pura Taman Ayun,"Religius, Badung, Dekat dari Bandara, Murah",Pura Taman Ayun di susun dengan konsep taman t...
72,Pura Tirta Empul,"Religius, Gianyar, Sedang dari Bandara, Murah",Tirta Empul adalah nama sebuah pura Hindu yang...
73,Pura Uluwatu,"Religius, Badung, Dekat dari Bandara, Mahal",Pura yang terletak di ujung barat daya pulau B...


## Recommendation by Genre

In [8]:
df_Tags = df.tags
df_Tags

0      Agrowisata, Gianyar, Dekat dari Bandara, Mahal
1     Agrowisata, Gianyar, Sedang dari Bandara, Mahal
2     Agrowisata, Denpasar, Dekat dari Bandara, Murah
3     Agrowisata, Gianyar, Dekat dari Bandara, Gratis
4     Agrowisata, Tabanan, Sedang dari Bandara, Murah
                           ...                       
70      Religius, Gianyar, Dekat dari Bandara, Gratis
71        Religius, Badung, Dekat dari Bandara, Murah
72      Religius, Gianyar, Sedang dari Bandara, Murah
73        Religius, Badung, Dekat dari Bandara, Mahal
74     Religius, Karangasem, Jauh dari Bandara, Murah
Name: tags, Length: 75, dtype: object

In [9]:
def remove_brackets(data):
    data = str(data).lower()
    data = data.replace(',', '')
    data = data.replace("'", '')
    return data

df_Tags = df_Tags.apply(remove_brackets)
df_Tags

0      agrowisata gianyar dekat dari bandara mahal
1     agrowisata gianyar sedang dari bandara mahal
2     agrowisata denpasar dekat dari bandara murah
3     agrowisata gianyar dekat dari bandara gratis
4     agrowisata tabanan sedang dari bandara murah
                          ...                     
70      religius gianyar dekat dari bandara gratis
71        religius badung dekat dari bandara murah
72      religius gianyar sedang dari bandara murah
73        religius badung dekat dari bandara mahal
74     religius karangasem jauh dari bandara murah
Name: tags, Length: 75, dtype: object

In [10]:
vector_tfidf = TfidfVectorizer()
result_tfidf = vector_tfidf.fit_transform(df_Tags)

In [11]:
result_arr = result_tfidf.toarray()
result_arr

array([[0.61205958, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.58476617, 0.        , 0.        , ..., 0.        , 0.40172671,
        0.        ],
       [0.56503277, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.56580037, 0.46440488,
        0.        ],
       [0.        , 0.        , 0.47180427, ..., 0.54124189, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.44374903, 0.        ,
        0.        ]])

In [17]:
vector_tfidf.get_feature_names_out()

array(['agrowisata', 'alam', 'badung', 'bandara', 'bangli', 'belanja',
       'budaya', 'buleleng', 'dari', 'dekat', 'denpasar', 'gianyar',
       'gratis', 'jauh', 'karangasem', 'klungkung', 'mahal', 'murah',
       'pantai', 'religius', 'sedang', 'tabanan'], dtype=object)

In [20]:
# choose one or more category
test = 'pantai gratis'

In [27]:
test = 'alam dekat gratis'
test_arr = vector_tfidf.transform([test])
test_arr = test_arr.toarray()
test_arr

array([[0.        , 0.7061612 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.41267541,
        0.        , 0.        , 0.57535673, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [28]:
result = {}
for id, vector in enumerate(result_arr):
    cosine_val = cosine_similarity([result_arr[id]], test_arr)
    result[id] = cosine_val

result_desc = dict(sorted(result.items(), key=lambda item: item[1], reverse=True))

top_n = 5

for n, place_id in enumerate(result_desc):
    if n > 4:
        break
    print(df_filter.iloc[place_id]['place_name'])

Watersport Tanjung Benoa Bali
Air Terjun Tegenungan
Bali Bird Park
Monkey Forest Ubud
Bali Swing


##save data

In [23]:
vectortfidf_wisata = pickle.load(open('vector_tfidf.pkl','rb'))
vectortfidf_wisata

TfidfVectorizer()

In [13]:
# Save dataframe
pickle.dump(result_arr, open('resulttfidf_wisata.pkl', 'wb'))

In [16]:
resulttfidf_wisata = pickle.load(open('resulttfidf_wisata.pkl','rb'))
resulttfidf_wisata

array([[0.61205958, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.58476617, 0.        , 0.        , ..., 0.        , 0.40172671,
        0.        ],
       [0.56503277, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.56580037, 0.46440488,
        0.        ],
       [0.        , 0.        , 0.47180427, ..., 0.54124189, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.44374903, 0.        ,
        0.        ]])

In [30]:
pickle.dump(df, open('df.pkl', 'wb'))

In [22]:
pickle.dump(vector_tfidf, open('vector_tfidf.pkl', 'wb'))